### Final Project Submission

Please fill out:
* Student name: Brian Lee
* Student pace: Full time
* Scheduled project review date/time: --
* Instructor name: Victor
* Blog post URL: -- 

# Movie Analysis

## Business Problem

Summary of the business problem you are trying to solve, and the data questions that you plan to answer to solve them.

***

Questions to consider:

* What are the business's pain points related to this project?
* How did you pick the data analysis question(s) that you did?
* Why are these questions important from a business perspective?

In [ ]:
## Write thoughts here:

# Figure out what movies do well at the box office for Microsoft
#  * Movie Genre
#  * Does actors/directors play into "success"?
#  * Movie cost to return

## Using info from IMDB, Rotten Tomatoes, etc, figure out with data what will play into "success"
#  Use Data visualizations to help prove point

## Data Understanding

Describe the data being used for this project.

Movie datasets from: 

* Box Office Mojo
* IMDB
* Rotten Tomatoes 
* TheMovieDB
* The Numbers

Minimum datasets to use:

* imdb.title.basics
* imdb.title.ratings
* bom.movie_gross
***

Questions to consider:

* Where did the data come from, and how do they relate to the data analysis questions?
* What do the data represent? Who is in the sample and what variables are included?
* What is the target variable?
* What are the properties of the variables you intend to use?

In [1]:
# Import standard packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import sqlite3

%matplotlib inline

### IMDB data

In [37]:
# May be good to query the databases with sqlite3 as they contain various keys to connect various tables (after clearning data)
title_basics = pd.read_csv('zippedData/imdb.title.basics.csv.gz')
title_basics.head()
title_basics.isna().sum()

tconst                 0
primary_title          0
original_title        21
start_year             0
runtime_minutes    31739
genres              5408
dtype: int64

In [9]:
title_ratings = pd.read_csv('zippedData/imdb.title.ratings.csv.gz')
title_ratings.head()
# len(title_ratings)

,tconst,averagerating,numvotes
0,tt10356526,8.3,31
1,tt10384606,8.9,559
2,tt1042974,6.4,20
3,tt1043726,4.2,50352
4,tt1060240,6.5,21


In [25]:
# Try to merge title_ratings and basics
# title_basics.merge(title_ratings, how="inner", on='tconst').head()
imdb_df = title_basics.merge(title_ratings, how="inner", on='tconst')
# display(imdb_df.describe())
# missing data in runtime_minutes and genres. We probably want to remove items without genre as it is defining trait
# imdb_df['genres'].unique()
imdb_df.dropna(subset=['genres'], inplace=True)
imdb_df.isna().sum()

tconst                0
primary_title         0
original_title        0
start_year            0
runtime_minutes    7332
genres                0
averagerating         0
numvotes              0
dtype: int64

In [5]:
## Table of actors/directors/writers - joined with imdb
# Need to clean for NaN values for characters or id (if we will use)

title_crew = pd.read_csv('zippedData/imdb.title.principals.csv.gz')
title_crew.head()
# len(title_crew)

,tconst,ordering,nconst,category,job,characters
0,tt0111414,1,nm0246005,actor,NaN,"[""The Man""]"
1,tt0111414,2,nm0398271,director,NaN,NaN
2,tt0111414,3,nm3739909,producer,producer,NaN
3,tt0323808,10,nm0059247,editor,NaN,NaN
4,tt0323808,1,nm3579312,actress,NaN,"[""Beth Boothby""]"


### Box Office Mojo

In [32]:
movie_gross = pd.read_csv('zippedData/bom.movie_gross.csv.gz')
movie_gross.tail()
display(len(movie_gross))
movie_gross.isna().sum()

# around 1/3 of foreign_gross is missing.
# should do a separate analysis for domestic/foreign when graphing. domestic/foreign success is not 100% correlated

3387

title                0
studio               5
domestic_gross      28
foreign_gross     1350
year                 0
dtype: int64

### Rotten Tomatoes data

In [33]:
movie_info = pd.read_csv('zippedData/rt.movie_info.tsv.gz', delimiter="\t")
movie_info.head()
movie_info.isna().sum()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [37]:
# try:
#     movie_reviews = pd.read_csv('zippedData/rt.reviews.tsv.gz', delimiter="\t")
#     movie_reviews.head()
# except Exception as e:
#     print(e)

In [35]:
rt_movie_info_df=pd.read_csv('zippedData/rt.movie_info.tsv.gz', sep='\t', header=0)
rt_movie_info_df.head()

,id,synopsis,rating,genre,director,writer,theater_date,dvd_date,currency,box_office,runtime,studio
0,1,"This gritty, fast-paced, and innovative police...",R,Action and Adventure|Classics|Drama,William Friedkin,Ernest Tidyman,"Oct 9, 1971","Sep 25, 2001",NaN,NaN,104 minutes,NaN
1,3,"New York City, not-too-distant-future: Eric Pa...",R,Drama|Science Fiction and Fantasy,David Cronenberg,David Cronenberg|Don DeLillo,"Aug 17, 2012","Jan 1, 2013",$,"600,000",108 minutes,Entertainment One
2,5,Illeana Douglas delivers a superb performance ...,R,Drama|Musical and Performing Arts,Allison Anders,Allison Anders,"Sep 13, 1996","Apr 18, 2000",NaN,NaN,116 minutes,NaN
3,6,Michael Douglas runs afoul of a treacherous su...,R,Drama|Mystery and Suspense,Barry Levinson,Paul Attanasio|Michael Crichton,"Dec 9, 1994","Aug 27, 1997",NaN,NaN,128 minutes,NaN
4,7,NaN,NR,Drama|Romance,Rodney Bennett,Giles Cooper,NaN,NaN,NaN,NaN,200 minutes,NaN


In [34]:
rt_reviews_df=pd.read_csv('zippedData/rt.reviews.tsv.gz',sep='\t',header=0, encoding='latin-1')
rt_reviews_df.head()

,id,review,rating,fresh,critic,top_critic,publisher,date
0,3,A distinctly gallows take on contemporary fina...,3/5,fresh,PJ Nabarro,0,Patrick Nabarro,"November 10, 2018"
1,3,It's an allegory in search of a meaning that n...,NaN,rotten,Annalee Newitz,0,io9.com,"May 23, 2018"
2,3,... life lived in a bubble in financial dealin...,NaN,fresh,Sean Axmaker,0,Stream on Demand,"January 4, 2018"
3,3,Continuing along a line introduced in last yea...,NaN,fresh,Daniel Kasman,0,MUBI,"November 16, 2017"
4,3,... a perverse twist on neorealism...,NaN,fresh,NaN,0,Cinema Scope,"October 12, 2017"


### The Numbers data

In [40]:
## Movie budgets - The Numbers
# Has production budget, domestic/worldwide gross

movie_budgets = pd.read_csv('zippedData/tn.movie_budgets.csv.gz')
movie_budgets.head()
# movie_budgets.isna().sum() # no missing values

,id,release_date,movie,production_budget,domestic_gross,worldwide_gross
0,1,"Dec 18, 2009",Avatar,"$425,000,000","$760,507,625","$2,776,345,279"
1,2,"May 20, 2011",Pirates of the Caribbean: On Stranger Tides,"$410,600,000","$241,063,875","$1,045,663,875"
2,3,"Jun 7, 2019",Dark Phoenix,"$350,000,000","$42,762,350","$149,762,350"
3,4,"May 1, 2015",Avengers: Age of Ultron,"$330,600,000","$459,005,868","$1,403,013,963"
4,5,"Dec 15, 2017",Star Wars Ep. VIII: The Last Jedi,"$317,000,000","$620,181,382","$1,316,721,747"


## Data Preparation

Describe and justify the process for preparing the data for analysis.

***

Questions to consider:

* Were there variables you dropped or created?
* How did you address missing values or outliers?
* Why are these choices appropriate given the data and the business problem?

In [ ]:
# Here you run your code to clean the data

## Data Modeling

Describe and justify the process for analyzing or modeling the data.

***

Questions to consider:
* How did you analyze or model the data?
* How did you iterate on your initial approach to make it better?
* Why are these choices appropriate given the data and the business problem?

In [ ]:
# Here you run your code to model the data

## Evaluation

Evaluate how well your work solves the stated business problem.

***

Questions to consider:

* How do you interpret the results?
* How well does your model fit your data? How much better is this than your baseline model?
* How confident are you that your results would generalize beyond the data you have?
* How confident are you that this model would benefit the business if put into use?

In [ ]:
# Write thoughts here:

## Conclusions

Provide your conclusions about the work you've done, including any limitations or next steps.

***

Questions to consider:

* What would you recommend the business do as a result of this work?
* What are some reasons why your analysis might not fully solve the business problem?
* What else could you do in the future to improve this project?

In [ ]:
# Write thoughts here:

## Next Steps: